In [1]:
import time

class Timer:
    START = 'start'
    END = 'end'
    
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.current_time = None
        self.events = []
        self.duration = None
    
    def start(self):
        t = time.time()
        self.start_time = t
        self.current_time = t
        self._add_event(t, self.START)
    
    def end(self):
        t = time.time()
        self.end_time = t
        self._add_event(t, self.END)
        self.duration = self.end_time - self.start_time
        
    def log_event(self, event_name):
        self._add_event(time.time(), event_name)
        self.current_time = time.time()
    
    def _add_event(self, event_time, event_name):
        if len(self.events) > 0:
            duration = event_time - self.current_time
            self.events[-1][-1] = duration
        
        self.events.append([event_name, event_time, 0])

In [2]:
import cv2

def tag_frame(frame, bounds, name=None, bounding_box_color=(0,0,255), text_color=(255,255,255)):
    box_width = 2
    top, right, bottom, left = bounds
    cv2.rectangle(frame, (left, top), (right, bottom), bounding_box_color, box_width)
    
    if name is not None:
        text_offset = 15
        text_padding = box_width
        cv2.rectangle(frame, (left, bottom), (right, bottom + text_offset + text_padding), bounding_box_color, -1)
        cv2.putText(frame, name.upper(), (left + text_padding, bottom + text_offset - text_padding), cv2.FONT_HERSHEY_DUPLEX, 0.5, text_color, 1)
        
    return frame

def tag_faces_in_video_old(input_path, output_path, tag_data):
    vreader = skvideo.io.vreader(input_path)
    vwriter = skvideo.io.FFmpegWriter(output_path)
    
    for i, frame in enumerate(vreader):
        if i % 1000 == 0:
            print(f'Processing frame {i}')
        
        output_frame = np.copy(frame)
        for label, bounds in tag_data[i]:
            tag_frame(output_frame, bounds, name=label)
        
        vwriter.writeFrame(output_frame)
            
    vwriter.close()
    
def tag_faces_in_video(input_path, output_path, datastore):
    vreader = skvideo.io.vreader(input_path)
    vwriter = skvideo.io.FFmpegWriter(output_path)
    
    for i, frame in enumerate(vreader):   
        face_data = datastore.face_data.setdefault(i, [])
        for fd in face_data:
            tag_frame(frame, fd.bounds, name=fd.face)
        vwriter.writeFrame(frame)
            
    vwriter.close()
    


In [3]:
import skvideo.io
import os

class Video:
    def __init__(self, path):
        metadata = skvideo.io.ffprobe(path)['video']
        self.frame_rate = float(eval(metadata['@avg_frame_rate']))
        self.frames = skvideo.io.vread(path)
    
    def get_frames(self, start_frame, end_frame):
        return np.copy(self.frames[start_frame:end_frame])

    def get_segment(self, start_time, end_time):
        fpms = self.frame_rate / 1000
        start_frame = int(start_time * fpms)
        end_frame = int(end_time * fpms)
        return self.get_frames(start_frame, end_frame)
    
    def get_sub_segment(self, sub):
        return self.get_segment(sub.start_time, sub.end_time)
    
    def get_frame_range_for_sub(self, sub):
        fpms = self.frame_rate / 1000
        start_frame = int(sub.start_time * fpms)
        start_frame = min(start_frame + 1, len(self.frames) - 1)
        
        end_frame = int(sub.end_time * fpms)
        end_frame = max(end_frame - 1, 0)
        return start_frame, end_frame